# Prototypes of real-time analysis system for neural data

_Author_: Conor McGrory (conor.mcgrory@stonybrook.edu), rotation student

## Overview

This repository contains the code and notes from my rotation project in the Park Lab from September to December 2020. The goal of this project was to develop multiple prototypes of a real-time analysis system for neural electrophysiological data, using different platforms, in order to better understand both the technical challenges that need to be overcome in order to build such as system and the tools best suited for the problem. (The work on the C protoype was added later, in the spring of 2021).

### How to use the notebooks

The notebooks in this folder are meant to be read/run in order, and contain all of the code necessary for fetching required input data and analyzing experimental results, as well as detailed instructions for running the experiments themselves. 

### Notebook index

The notebooks in this directory are:

- [01_overview.ipynb](./01_overview.ipynb): The notebook you are currently reading. Contains a broad overview of the project and explains its results.
- [02_raw_data.ipynb](./02_raw_data.ipynb): Contains code for downloading and examining the data used for this project. 
- [03_test_signal.ipynb](./03_test_signal.ipynb): Contains code for compiling raw data into signal that is used to test the prototypes, and examining signal.
- [04_lms_filter.ipynb](./04_lms_filter.ipynb): Explores the properties of least-mean-squares (LMS) filtering, an adaptive filtering method used for the prototypes
- [05_results_python.ipynb](./05_results_python.ipynb): Contains instructions for running latency experiment with Python prototype, along with code for analyzing results
- [06_results_julia.ipynb](./06_results_julia.ipynb): Contains instructions for running latency experiment with Julia prototype, along with code for analyzing results
- [07_results_rust.ipynb](./07_results_rust.ipynb): Contains instructions for running latency experiment with Rust prototype, along with code for analyzing results
- [08_results_c.ipynb](./08_results_c.ipynb): Contains instructions for running latency experiment with C prototype, along with code for analyzing results
- [09_results_compare.ipynb](./08_results_compare.ipynb): Contains code for generating plots comparing speed of prototypes, some of which appear later in this notebook

## Background: Real-time analysis of neural data

The broad research objective that motivates this project is the development of a real-time analysis system for electrophysiological data. Being able to process neural data in real-time during the course of an experiment would enable us to perform a whole new class of experimental manipulations, including closed-loop experiments and custom perturbations of neural circuits.

![filter setup](img/filter_setup.png)

Our proposed setup uses two computers. The first, called the "probe" machine, interfaces directly with the probe used to collect the data (in our case, this will most likely be the [Neuropixel](https://www.neuropixels.org/)). At each time step, it reads data from the probe and sends it to the second computer, called the "processor," which uses the incoming data to update an adaptive filter or some other type of online data analysis method. In the case of a closed-loop experiment, the "processor" machine will send data back to the "probe" machine that will change some element of the experiment in response to a particular state of the filter.

One of the most significant technical challenges that needs to be overcome in order to make this system possible is the problem of _latency_. Modern probes, such as the Neuropixel, record data at very high sample rates, which in theory allows us to perform closed-loop experiments with a high level of temporal precision. However, the limiting factor currently preventing this is the time it takes for the analysis method to run. The broad aim of this project was to build simple prototypes of the real-time analysis system on a number of different platforms in order to understand how to minimize latency.

## Objectives

The main objectives for this project were:
1. Build simple prototypes of the real-time analysis system using four different languages: Python, Julia, Rust, and C
2. Measure the latency properties of all three prototypes and compare them


## Prototype setup

![prototype](./img/prototype.png)

The basic structure of the prototype is set up to mimic the conditions of an actual experiment as well as we can _in silico_. The probe and processor machines are simulated using two processes, which can either be run on two separate machines connected via Ethernet, or on a single machine using the local link hardware. In this setup, the processor machine functions as a server process, and the probe as a client. The processor, when run, waits for the probe to connect using a TCP/IP socket on a prespecified port. Once the probe is connected, it starts sending the test data, which consists of vectors of spike counts, across the socket, to the processor, which returns a "filter prediction" for each data point. The test data (see below) is loaded from an HDF5 file before the data transmission begins, in order to ensure that all measured latency is the result of the prototype itself.

When transmitting the data, the probe sends the spike count vectors one at a time, waiting until it recieves a response from the processor before it sends another one. While this "blocking" approach would clearly be suboptimal if used in a real experiment, it allows us to measure the round-trip latency -- the elapsed time between sending a spike vector and recieving its filter prediction -- in a straightforward and accurate way. In future prototypes, this approach will be replaced by an approach using two nonblocking sockets: one for sending spike data from probe to processor, and the other for feedback data from processor to probe.

### Test data

![raw data](./img/raw_data.png)

The neural data we use to test our prototypes is a publicly available dataset from [Steinmetz et al., 2019](https://www.nature.com/articles/s41586-019-1787-x). This data contains 698 neurons, and uses a bin size of 10 ms. Individual trials are 2.5 sec long, and contain 250 data points. The notebook [02_raw_data.ipynb](./02_raw_data.ipynb) contains the code used to download this data from the server and examine it locally.

Because the trials are relatively short, we concatenate all of the trials for a particular session into a longer signal, called the "test signal", for use with the prototypes. This is explained further in [03_test_signal.ipynb](./03_test_signal.ipynb), which contains the code used to create this signal. 

### Least-mean-squares filter

For the processor component of the prototypes, we used a simple adaptive filter called a [least-mean-squares (LMS) filter](https://en.wikipedia.org/wiki/Least_mean_squares_filter). Because this filter is linear, it isn't likely to be useful for any actual neural data analysis, but its ease of implementation and relatively low computational cost made it a good algorithm to use for measuring latency in different prototypes. For an exploration of LMS filtering and its application to our test data, see [04_lms_filter.ipynb](./04_lms_filter.ipynb).

![lms](./img/lms_synthetic.png)

### More complex filters

The initial goal of this project was to implement a more complex filter, like a Kalman filter, in the prototypes, to see what the latency of our system would be when using the kind of filter we would actually need to extract useful information from neural data. However, because I ended up spending more time than I expected decreasing the latency of the prototypes using the LMS filter, this will need to be left for future work.

### Choice of programming languages

The reason I ended up creating prototypes of the real-time analysis system in three different programming languages was because, due to the nature of the task, there was no obvious best choice for a language to use. Offline data analysis tasks are usually best accomplished using a high-level language such as Python or MATLAB, which allows researchers to quickly and easily code up different analysis pipelines without worrying about low-level issues like memory management. The fact that these languages are significantly slower compared to more low-level languages like C or Fortran isn't an issue, because the time constraints for these tasks are relatively loose. For our task, however, speed is extremely important -- in order for our method to be experimentally useful, it needs to be able to analyze data as fast as it is collected. This puts us in a more tough position then: we need the speed that comes with low-level languages, but we also need to be able to quickly and reliably implement somewhat complex filtering operations to run on the data.

The first language I used to build a prototype of the real-time analysis system was [Julia](https://julialang.org/). This high-level language was attractive for a number of reasons, the chief among them being that because of its type system and just-in-time (JIT) compiler, it can perform certain benchmark computational tasks almost as fast as C, while being much easier to program in. The network programming functionality ended up being difficult to work with, and caused some early latency problems that were only resolved recently.

In order to compare Julia against a purely high-level language, I wrote the second prototype in [Python](https://www.python.org/). This prototype was easy to build, and because of the straightforward interface for network programming, and the use of NumPy for the filtering operation, it worked decently well. 

In addition, I wanted to try using a low-level language, to see if this could decrease the latency even more. While C is a much more established language, I ended up first trying [Rust](https://www.rust-lang.org/) because of its memory management features and a third-party linear algebra library called [ndarray](https://docs.rs/ndarray/0.14.0/ndarray/) that supports a NumPy-style interface for matrix computations. 

However, as is explained further below, the performance gains from using Rust were not as significant as expected, so I ended up building a prototype in C, using the [OpenBLAS](https://www.openblas.net/) library for matrix computations.

Below is a table showing the strengths and weaknesses of the four languages used for the prototypes:

|                             | Python          | Julia     | Rust           | C              |
| --------------------------- | --------------- | --------- | -------------- | -------------- |
| Learning curve              | Very easy       | Very easy | More difficult | More difficult |
| Speed                       | Relatively slow | Faster    | Fastest        | Fastest        |
| Numerical computing support | Very good       | Very good | Some           | Good           |
| Network support             | Good            | Good      | Good           | Good           |
| Application support         | Good            | Not good  | Good           | Good           |

## Results

### Echo filter latencies

![echo latencies](./img/echo_latencies.png)

As this plot clearly shows, the echo filter latency distributions of the Python and Rust prototypes are very similar, while the C prototype is slightly fast and the Julia prototype is slightly slower. The median latency for the C prototype is 29 $\mu s$, the median latency for the Python prototype is 45 $\mu s$, the median latency for the Rust prototype is 37 $\mu s$, and the median latency for the Julia prototype is 110 $\mu s$. In some ways, this is what we would expect -- the echo filter doesn't process the data at all, so the only sources of latency here are the socket overhead, which, being a kernel call, should be similar for all prototypes, and the cost associated with encoding data into bytes to send it across the socket and decoding it on the other end, which should be minimal. However, it is unclear why the Julia prototype is slower. One likely explanation would be that it has something to do with either how the data is encoded before sending it over the socket, or the Julia network API itself, which seems to be less direct to the kernel than the C, Python, or Rust socket APIs. This is an important result because it establishes that any latency differences we see with the LMS filter are the caused by the code that implements the filter and nothing else.

### LMS filter latencies

![lms latencies](./img/lms_latencies.png)

In this plot of the LMS filter latency distributions of the Python, Rust, and C prototypes, it is clear that the C prototype is the fastest by a significant amount, followed by the Python and then the Rust prototype. This is also reflected in the median values (C: 1531 $\mu s$, Python: 5579 $\mu s$, Rust: 11174 $\mu s$). Because the echo filter latencies of all of these prototypes were extremely close, this difference is almost certainly due to the LMS filter code. 

The fact that the C prototype is fastest is hardly surprising. The C code is directly calling the BLAS functions that the Python implementation calls under the hood, and manages memory in a more efficient way.

There are two explanations I can think of for the difference between the Python and Rust prototypes. The first is that the [ndarray](https://docs.rs/ndarray/0.14.0/ndarray/) package that the Rust prototype uses to implement the LMS filter is slower than NumPy -- this could be caused by differences in how the two packages call out to the LAPACK or BLAS libraries for matrix computations. The other possiblity is that the way the LMS filter code in the Rust prototype is written is inefficient in terms of memory allocation. This is definitely possible, considering that I had no prior experience programming in Rust before working on this project, and still have some issues working with its memory management system.

## Conclusions

Looking at both the LMS and echo filter data, the C prototype is clearly the fastest of the three, and the Python prototype the second fastest. With sufficient optimization, the Rust and Julia prototypes might also achieve similar speeds as the Python prototype, or perhaps be even faster. As it currently stands, the bottleneck for the Julia prototype is the problem of transferring data over the socket, which has been hard to solve both because Julia's socket interface doesn't directly match kernel calls, and because there isn't much documentation available on it. The bottleneck for the Rust prototype is the speed of the LMS filter, which can probably be optimized more, either by using a different linear algebra library, or examining the memory trace of the code more carefully. 

One important thing I need to note here is that it's probably not a coincidence that the prototype that currently works the second-best is the one written in the language I have the most personal experience with. Before starting this project, I had never written a line of Julia or Rust, but I have been programming in Python for 5 years. Julia's network API and Rust's memory management systems were both hard for me to figure out at first, and I wouldn't be surprised at all if the code I wrote in those languages still contains some "rookie mistakes".

The only real conclusion to draw from this data is that **C is the best language to use for building a usable real-time filtering system**. The average latency achieved by the prototype, around 1.5 ms, is definitely fast enough to be used in the context of a real experiment. However, it's important to remember that almost all of the latency in all of these prototypes comes from the matrix calculations performed by the filter, and that the least-mean-squares filter is relatively fast compared to some other types of filters.